In [1]:
macro javascript_str(s) display("text/javascript", s); end
javascript"""
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } },
    "HTML-CSS": { 
        availableFonts: ["TeX"],
        preferredFont: "TeX"
    }
});
"""

# DQMC (Determinant Quantum Monte-Carlo) Method


This notebook is **not** completely finished. **DO NOT** upload or share it publicly.

This notebook may contain errors and bugs. If you have found one, feel free to [send an email](mailto:chli@iphy.ac.cn) to tell me.


## Prequites

Hamitonian for Hubbard model:

$$
\begin{equation}
H = -t\sum_{<ij>,\sigma}c^\dagger_{i\sigma} c_{j\sigma} + h.c. + U\sum_{i}(n_{i\uparrow}-\frac{1}{2}) (n_{i\downarrow}-\frac{1}{2})
\end{equation}
$$


The partition function
$$
\begin{equation}
    \DeclareMathOperator{\Tr}{Tr}
    Z = \Tr\left[e^{\beta H}\right] \\
      = \Tr\left[e^{-\varDelta\tau \cdot M \cdot H}\right] \\
      = \Tr\left[(e^{-\varDelta\tau \cdot H}\right)^M]
\end{equation}
$$
$\beta = -\varDelta\tau \cdot M $

Do Trotter Decomposition
$$
\begin{equation}
    e^{-\varDelta\tau H} \approxeq e^{-\varDelta\tau H_I}e^{-\varDelta\tau T}
\end{equation}
$$
in which $T = -t\sum_{<ij>,\sigma}c^\dagger_{i\sigma} c_{j\sigma}$, $H_I = U(n_{i\uparrow}-\frac{1}{2}) (n_{i\downarrow}-\frac{1}{2})$

For term with $H_I$, we do [Hubbard–Stratonovich transformation](https://en.wikipedia.org/wiki/Hubbard%E2%80%93Stratonovich_transformation), then we get

$$
\begin{equation}
    e^{-\varDelta\tau H_I} = \prod_{i} e^{\varDelta\tau U\sum_{i}(n_{i\uparrow}-\frac{1}{2}) (n_{i\downarrow}-\frac{1}{2})} \\
    = \prod_{i} \lambda \sum_{s_i,\tau=\pm 1} e^{\alpha s_{i,\tau}(n_{i,\uparrow} - n_{i,\downarrow})}\\
    = \lambda^{N} \sum_{s_i,\tau=\pm 1} ( \prod_{i} e^{\alpha s_{i,\tau}n_{i,\uparrow}} \prod_{i} e^{-\alpha s_{i,\tau} n_{i,\downarrow}})
\end{equation}
$$

For term with $T$
$$
\begin{equation}
    e^{-\varDelta\tau T} = \prod_{\sigma} e^{-\varDelta\tau t \sum_{<ij>,\sigma} (c^\dagger_{i\sigma} c_{j\sigma} + c^{\dagger}_{j\sigma}c_{i\sigma})}
\end{equation}
$$

After a series of mathematical calculation (mentioned in the lecture before), we can finally write the partition function into a determinant form

$$
\begin{equation}
    Z = \lambda^{MN} \sum_{s_i, \tau} \prod_{\sigma} \det \left[ I + B_{M}^{\sigma}B_{M-1}^{\sigma}B_{M-2}^{\sigma}\cdots B_{2}^{\sigma}B_{1}^{\sigma} \right]
\end{equation}
$$

In the above equation, the $s_i$ is the lattice site index, $\tau$ is the imaginary time index (from $M$ to $1$), $\sigma$ is the spin orientation index.

## Calculation

At the beginning, we should import the necessary library: `LinearAlgebra`, `Random`, `Statistics`

In [2]:
using LinearAlgebra
using Random
using Statistics

Here we put some help function

In [3]:
function Diag(Mat)
    LinearAlgebra.Diagonal(LinearAlgebra.eigvals(Mat))
end

function Spin_Diag_list(time_slices::Int, lattice_list::Array)
    S_mat_list = Array[]
    for i = 1:time_slices
        push!(S_mat_list, Spin_Diag(lattice_list[i]))
    end
    S_mat_list
end

function Spin_Diag(lattice::Array)
    spin_diag = zeros(Int8, length(lattice), length(lattice))
    i = 1
    for spin in lattice
        spin_diag[i, i] = spin
        i = i + 1
    end
    return spin_diag
end

Spin_Diag (generic function with 1 method)

We should setting the parameter that used in simulation

In [4]:
l = 4::Int     # System Size
t = 1.0        # Hopping amplitude, t in the Hamitonian
U = 1.0        # U in Hubbard Model Hamitonian
β = 4::Int     # Inverse of temperature, β in the Hamitonian
Δτ = 0.05      # Imaginary time slice size
μ = 0          # Chemical potential

α = acosh(exp(0.5*Δτ*U))

N_site = l * l
time_slices = Int(β / Δτ)

N_warm_up_sweep = 4 * N_site * β
N_sweep_per_bin = 200
N_bin = 5
N_wrap = 10

10

Generate square lattice list, for every time slice we need to generate a 2-d lattice with each site in the lattice is set to either -1 or 1.

In [6]:
struct Neighbor
   	up::Int
   	down::Int
   	left::Int
   	right::Int
end

function init_lattice_list(time_slices::Int, size::Int)
   	lattice_list = Array[]
   	for ts = 1:time_slices
       	push!(lattice_list, init_lattice(size))
   	end
   	lattice_list
end

function init_lattice(size::Int)
    lattice = rand([-1,1], size, size)
    lattice
end

init_lattice (generic function with 1 method)

Here we need to generate a list to store the index of neighbor in the lattice.

In [7]:
function gen_neighbor_list(lattice::Array{}, size::Int)
    neighbor_index_list = Neighbor[]
    lat_linear_iter = LinearIndices(lattice)
    for coord in CartesianIndices(lattice)
        row = coord[1]
        col = coord[2]

        next_row = row + 1
        next_col = col + 1
        prev_row = row - 1
        prev_col = col - 1
        if next_row > size
            next_row = 1
        end
        if next_col > size
            next_col = 1
        end
        if prev_row < 1
            prev_row = size
        end
        if prev_col < 1
            prev_col = size
        end
        up_cart_coord = CartesianIndex(prev_row, col)
        down_cart_coord = CartesianIndex(next_row, col)
        left_cart_coord = CartesianIndex(row, prev_col)
        right_cart_coord = CartesianIndex(row, next_col)
        neighbor_index = Neighbor(lat_linear_iter[up_cart_coord], lat_linear_iter[down_cart_coord], lat_linear_iter[left_cart_coord], lat_linear_iter[right_cart_coord])
        push!(neighbor_index_list, neighbor_index)
    end
    return neighbor_index_list
end

gen_neighbor_list (generic function with 1 method)

Calculate $T$ matrix
$$
\begin{equation}
e^{-\varDelta\tau T} = \prod_{\sigma} e^{-\varDelta\tau t \sum_{<ij>,\sigma} (c^\dagger_{i\sigma} c_{j\sigma} + c^{\dagger}_{j\sigma}c_{i\sigma})}
\end{equation}
$$

For Hubbard model, we have
$$
\begin{equation}
    T_{i,j=\uparrow, \downarrow ,\leftarrow, \rightarrow} = t
\end{equation}
$$
$\uparrow$, $\downarrow$ ,$\leftarrow$, $\rightarrow$ indicates the neighbor's relative direction.

In [8]:
function T(t, lattice_size, lattice::Array{}, neighbor_index_list::Array{Neighbor,1})
    T_mat = zeros(Float64, length(lattice), length(lattice))
    for i = 1:(lattice_size * lattice_size)
        T_mat[i, neighbor_index_list[i].up] = t
        T_mat[i, neighbor_index_list[i].down] = t
        T_mat[i, neighbor_index_list[i].left] = t
        T_mat[i, neighbor_index_list[i].right] = t
    end
    T_mat
end

T (generic function with 1 method)

Calculate the $B_{M}^{\sigma}$ matrix

In [9]:
function B_up(α, S, exp_A_T_mat)
    B_up_mat = exp(α * S) * exp_A_T_mat
    B_up_mat
end

function B_down(α, S, exp_A_T_mat)
    B_down_mat = exp(-α * S) * exp_A_T_mat
    B_down_mat
end

B_down (generic function with 1 method)

Calculate imaginary time propagators $B^{\sigma}(\tau_2, \tau_1)$ matrix. $B^{\sigma}(\tau_2, \tau_1)$ is calculate with
$$
\begin{equation}
    B^{\sigma}(\tau_2,\tau_1) = B(\tau_2)\cdots B(\tau_1)
\end{equation}
$$

In [10]:
function B_up_range(stop_time_index::Int, start_time_index::Int, α, B_up_mat_list)
    B_up_prop = zeros(N_site, N_site)
    for i = 1:N_site
        B_up_prop[i,i] = 1
    end
    for i = start_time_index + 1:stop_time_index
        B_up_prop = B_up_mat_list[i] * B_up_prop
    end
    B_up_prop
end

function B_down_range(stop_time_index::Int, start_time_index::Int, α, B_down_mat_list)
    B_down_prop = zeros(N_site, N_site)
    for i = 1:N_site
        B_down_prop[i,i] = 1
    end
    for i = start_time_index + 1:stop_time_index
       	B_down_prop = B_down_mat_list[i] * B_down_prop
    end
    B_down_prop
end

B_down_range (generic function with 1 method)

Calculate the Green function, defined as $G^{\sigma}(\tau,\tau) = (I + B^{\sigma}(\tau,0)B^{\sigma}(\beta,\tau))^{-1}$, $\tau$ indicates the imaginary time slice index.

We choose to use SVD decomposition to calculate the Green function. The reason is that when the $U$ is large, the matrix $I + B^{\sigma}(\tau,0)B^{\sigma}(\beta,\tau)$ will be more similar to the singular matrix, which causes the inaccuracy of the matrix inverse. In order to deal with the problem, we use SVD decomposition to separate the big and small part of the eigen value. 

$$
\begin{equation}
    G = (I + B(\tau,0)B(\beta,\tau))^{-1}\\
    = (I + U_R D_R V_R V_L D_L U_L)^{-1} \\
    = U_{L}^{-1} (D_{L}^{max})^{-1} [(D_{R}^{max})^{-1}(U_R U_L)^{-1} (D_{L}^{max})^{-1} + D_R^{min}V_R V_L D_{L}^{min}]^{-1} (D_{R}^{max})^{-1} U_R^{-1}
\end{equation}
$$

$D_{R}^{max}$ is a diagonal matrix derived from $D_{R}$, with the number **smaller** than 1 replaces by 1. $D_{R}^{min}$ is a diagonal matrix derived from $D_{R}$, with the number **larger** than 1 replaces by 1.

In [11]:
function G_up_τ_τ(time_slices::Int, current_time_index::Int, α, B_up_mat_list)
    R = B_up_range(current_time_index, 0, α, B_up_mat_list)
    L = B_up_range(time_slices, current_time_index, α, B_up_mat_list)

    R_UDV = LinearAlgebra.svd(R)
    L_UDV = LinearAlgebra.svd(L)

    g = G_σ_τ_τ_calc(R_UDV, L_UDV, N_site)
    g
end

function G_down_τ_τ(time_slices::Int, current_time_index::Int, α, B_down_mat_list)
    R = B_down_range(current_time_index, 0, α, B_down_mat_list) 
    L = B_down_range(time_slices, current_time_index, α, B_down_mat_list)

    R_UDV = LinearAlgebra.svd(R)
    L_UDV = LinearAlgebra.svd(L)
    
    g = G_σ_τ_τ_calc(R_UDV, L_UDV, N_site)
    g
end

function G_σ_τ_τ_calc(R_UDV, L_UDV, N_site)
    U_R = R_UDV.U
    V_R = R_UDV.Vt

    V_L = L_UDV.U
    U_L = L_UDV.Vt
    D_R_max = zeros(N_site)
    D_R_min = zeros(N_site)
    D_L_max = zeros(N_site)
    D_L_min = zeros(N_site)
    for i = 1:N_site
        if R_UDV.S[i] > 1
            D_R_max[i] = R_UDV.S[i]
            D_R_min[i] = 1
        else
            D_R_min[i] = R_UDV.S[i]
            D_R_max[i] = 1
        end
        if L_UDV.S[i] > 1
            D_L_max[i] = L_UDV.S[i]
            D_L_min[i] = 1
        else
            D_L_min[i] = L_UDV.S[i]
            D_L_max[i] = 1
        end
    end

    D_R_max_inv = LinearAlgebra.Diagonal(1 ./ D_R_max)
    D_L_max_inv = LinearAlgebra.Diagonal(1 ./ D_L_max)

    G_σ_τ_τ = inv(U_L) * D_L_max_inv * inv(D_R_max_inv * inv(U_L * U_R) * D_L_max_inv + LinearAlgebra.Diagonal(D_R_min) * V_R * V_L * LinearAlgebra.Diagonal(D_L_min)) * D_R_max_inv * inv(U_R)
    
    G_σ_τ_τ
end

G_σ_τ_τ_calc (generic function with 1 method)

Calculate the update ratio

$$
\begin{equation}
    R = \prod_{\sigma} \det \left[ I + \Delta^{\sigma}(i,r) G^{\sigma}(\tau,\tau) \right] \\
     = \prod_{\sigma} (I + \Delta^{\sigma}_{ii}(i,\tau)(I - G_{ii}^{\sigma}(\tau,\tau)))
\end{equation}
$$

$i$ is the lattice list linear index, $\tau$ is imaginary time index, $\sigma$ is spin orientation index. The $\Delta$ is given by the below formula

$$
   \begin{equation}
      \Delta = e^{\alpha  \mathrm{Diag}[S']} e^{-\alpha \mathrm{Diag}[S]} - I
   \end{equation}
$$

In [12]:
function Δ_up(coordinate, time_index::Int, α, lattice_list)
    after_flip = copy(lattice_list[time_index])
    after_flip[coordinate] = -after_flip[coordinate]
    exp(α * Spin_Diag(after_flip)) * exp(-α * Spin_Diag(lattice_list[time_index])) - LinearAlgebra.I
end

function R_up(Δ_up_mat, G_up_τ_τ_mat, lattice_index)
    1 + Δ_up_mat[lattice_index, lattice_index] * (1 - G_up_τ_τ_mat[lattice_index, lattice_index])
end

function Δ_down(coordinate, time_index::Int, α, lattice_list)
    after_flip = copy(lattice_list[time_index])
    after_flip[coordinate] = -after_flip[coordinate]
    exp(-α * Spin_Diag(after_flip)) * exp(α * Spin_Diag(lattice_list[time_index])) - LinearAlgebra.I
end

function R_down(Δ_down_mat, G_down_τ_τ_mat, lattice_index)
    1 + Δ_down_mat[lattice_index, lattice_index] * (1 - G_down_τ_τ_mat[lattice_index, lattice_index])
end

R_down (generic function with 1 method)

## Monte-Carlo Sampling

The key idea behind the Monte-Carlo Sampling is Markov Chain Monte-Carlo. In the sampling space, we randomly choose a star point, and let the system evolve, this is mapped to a movement in the sampling space. After some time, the system will finally go to a state that we called equivalent state. 

In order to fulfill the detailed equivalence condition, we should give the movement in the sampling space an acceptation probability, to determine whether to move or not. 

### Sweep

We iterate through the imaginary time slices, and in each time slices, we also iterate the site on the lattice. According to the spin on each site, we can the calculate the acceptation ratio. This ratio will compare to a random generated number to determine whether we update the Green function or not. 

A full process will last from $0$ to $\beta$, and then $\beta$ to $0$. This whole process was called a sweep.

In [19]:
function sweep(time_slices, green_up_list, green_down_list, B_up_mat_list, B_down_mat_list, lattice_list::Array, Spin_Diag_list, α, lattice_iterator, lattice_linear_iterator, measurement)
    sweep_range(time_slices, 1, green_up_list, green_down_list, B_up_mat_list, B_down_mat_list, lattice_list, Spin_Diag_list, α, lattice_iterator, lattice_linear_iterator, measurement)
    sweep_range(1, time_slices, green_up_list, green_down_list, B_up_mat_list, B_down_mat_list, lattice_list, Spin_Diag_list, α, lattice_iterator, lattice_linear_iterator, measurement)
end

function sweep_range(stop::Int, start::Int, green_up_list, green_down_list, B_up_mat_list, B_down_mat_list, lattice_list::Array, Spin_Diag_list, α, lattice_iterator, lattice_linear_iterator, measurement)

    range_iter = start:stop
    if stop < start
        range_iter = reverse(start:stop)
    end

    spin_updated = false
    previous_time_index = 1

    measure_time_index = rand(1:time_slices)

    for time_index in range_iter
        if time_index % N_wrap == 1
            recalc_green_function!(time_slices, time_index, α, B_up_mat_list, B_down_mat_list, green_up_list, green_down_list)
        else
            propagate_green_function!(previous_time_index, time_index, B_up_mat_list, B_down_mat_list, green_up_list, green_down_list)
        end
        global E_K_sum
        global DOC_sum
        global SF_sum
        
        if measurement && (time_index == measure_time_index)
            E_K_sum = E_K_sum +  kinetic_orig(t, green_up_list[time_index], green_down_list[time_index], nil, lattice_linear_iterator)
            DOC_sum = DOC_sum + double_occupancy_orig(green_up_list[time_index], green_down_list[time_index], lattice_linear_iterator)
            SF_sum = SF_sum + structure_factor(green_up_list[time_index], green_down_list[time_index], lattice_iterator, lattice_linear_iterator)
        end

        for coord in lattice_iterator
        # println("\t", coord)
            Δ_up_mat = Δ_up(coord, time_index, α, lattice_list)
            Δ_down_mat = Δ_down(coord, time_index, α, lattice_list)
            R_up_val = R_up(Δ_up_mat, green_up_list[time_index], lattice_linear_iterator[coord])
            R_down_val = R_down(Δ_down_mat, green_down_list[time_index], lattice_linear_iterator[coord])
            R = R_up_val * R_down_val

            counts[2] = counts[2] + 1

            if rand() < R
                lattice_list[time_index][coord] = -lattice_list[time_index][coord]
                Spin_Diag_list[time_index][lattice_linear_iterator[coord],lattice_linear_iterator[coord]] = -Spin_Diag_list[time_index][lattice_linear_iterator[coord],lattice_linear_iterator[coord]]

                update_green_up!(R_up_val, Δ_up_mat, time_index, green_up_list)
                update_green_down!(R_down_val, Δ_down_mat, time_index, green_down_list)

                counts[1] = counts[1] + 1
                spin_updated = true
            end
        end
        if spin_updated
            B_up_mat_list[time_index] = B_up(α, Spin_Diag_list[time_index], exp_A_T_mat)
            B_down_mat_list[time_index] = B_down(α, Spin_Diag_list[time_index], exp_A_T_mat)

            spin_updated = false
            # println("$time_index Update Spin")
            global B_updated = true
        end
        previous_time_index = time_index
    end
end

sweep_range (generic function with 1 method)

The Green function is updated with

$$
\begin{equation}
    G^{\sigma}(\tau,\tau)' = G^{\sigma}(\tau,\tau) - \frac{1}{R^{\sigma}} \Delta^{\sigma}(i,\tau) (I-G^{\sigma}(\tau,\tau))
\end{equation}
$$

In [14]:
function update_green_up!(R_up, Δ_up, time_index::Int, Green_up_list::Array)
    green_up_orig = Green_up_list[time_index]
    Green_up_list[time_index] = green_up_orig - (1 / R_up) * green_up_orig * Δ_up * (LinearAlgebra.I - green_up_orig)
end

function update_green_down!(R_down, Δ_down, time_index::Int, Green_down_list::Array)
    green_down_orig = Green_down_list[time_index]
    Green_down_list[time_index] = green_down_orig - (1 / R_down) * green_down_orig * Δ_down * (LinearAlgebra.I - green_down_orig)
end


update_green_down! (generic function with 1 method)

In theory we should calculate equal time Green function at each imaginary time slice with

$$
\begin{equation}
    G^{\sigma}(\tau,\tau) = (I + B^{\sigma}(\tau, 0)) B^{\sigma}(\beta,\tau))^{-1}
\end{equation}
$$

However, this will cause huge number of calculation and make the process slow. So, we choose to calculate an approximate equal time Green function with B propagator, and thus

$$
\begin{equation}
    G^{\sigma}(\tau+1,\tau+1) \approx B^{\sigma}(\tau+1,\tau)G^{\sigma}(\tau,\tau)B^{\sigma}(\tau+1,\tau)^{-1} \\
    G^{\sigma}(\tau-1,\tau-1) \approx B^{\sigma}(\tau,\tau-1)^{-1}G^{\sigma}(\tau,\tau)B^{\sigma}(\tau,\tau-1)
\end{equation}
$$

As the imaginary time increase, the accuracy of the formula is decrease. So, we should recalculate Green function with the accurate formula above.

In [15]:
function recalc_green_function!(time_slices, time_index, α, B_up_mat_list, B_down_mat_list, green_up_list, green_down_list)
    green_up_list[time_index] = G_up_τ_τ(time_slices, time_index, α, B_up_mat_list)
    green_down_list[time_index] = G_down_τ_τ(time_slices, time_index, α, B_down_mat_list)
end

function propagate_green_function!(previous_time_index, current_time_index, B_up_mat_list, B_down_mat_list, green_up_list, green_down_list)
    if previous_time_index > current_time_index
        B_up_τ1_τ = B_up_mat_list[current_time_index]# * B_up_mat_list[current_time_index]

        B_down_τ1_τ = B_down_mat_list[current_time_index]
        
        inv_B_up_τ1_τ = exp_mA_T_mat * exp(-α * S_list[current_time_index])
        inv_B_down_τ1_τ = exp_mA_T_mat * exp(α * S_list[current_time_index])
        green_up_list[current_time_index] = inv_B_up_τ1_τ * green_up_list[previous_time_index] * B_up_τ1_τ
        green_down_list[current_time_index] = inv_B_down_τ1_τ * green_down_list[previous_time_index] * B_down_τ1_τ
    else
        B_up_τ1_τ = B_up_mat_list[current_time_index] #* B_up_mat_list[previous_time_index]

        B_down_τ1_τ = B_down_mat_list[current_time_index] #* B_down_mat_list[previous_time_index]

        inv_B_up_τ1_τ = exp_mA_T_mat * exp(-α * S_list[current_time_index])
        inv_B_down_τ1_τ = exp_mA_T_mat * exp(α * S_list[current_time_index])
        green_up_list[current_time_index] = B_up_τ1_τ * green_up_list[previous_time_index] * inv_B_up_τ1_τ
        green_down_list[current_time_index] = B_down_τ1_τ * green_down_list[previous_time_index] * inv_B_down_τ1_τ
    end
end

propagate_green_function! (generic function with 1 method)

## Observable

The system kinetic energy is given by

$$
\begin{equation}
    T = - t \langle c_i^\dagger c_j \rangle
\end{equation}
$$

Double occupancy

$$
\begin{equation}
    D = \langle n_{i\uparrow} n_{j\downarrow} \rangle
\end{equation}
$$

Structure factor

$$
\begin{equation}
    S(Q) = \frac{1}{L^2}\sum_{ij} e^{-i Q \cdot (r_i - r_j)} <s_i s_j>
\end{equation}
$$

In [16]:
function kinetic_orig(t, green_up, green_down, neighbor_list, lattice_linear_iterator)
    E_K = 0
    green_up_c = LinearAlgebra.I - transpose(green_up)
    green_down_c = LinearAlgebra.I - transpose(green_down)
    for i in lattice_linear_iterator
        E_K = E_K + green_up_c[i, neighbor_list[i].up] + green_up_c[i, neighbor_list[i].down]
        E_K = E_K + green_up_c[i, neighbor_list[i].left] + green_up_c[i, neighbor_list[i].right]
        E_K = E_K + green_down_c[i, neighbor_list[i].up] + green_down_c[i,neighbor_list[i].down]
        E_K = E_K + green_down_c[i,neighbor_list[i].left] + green_down_c[i,neighbor_list[i].right]
    end
    t * E_K / N_site
end

function double_occupancy_orig(green_up, green_down, lattice_linear_iterator)
    doc = 0
    green_up_c = LinearAlgebra.I - transpose(green_up)
    green_down_c = LinearAlgebra.I - transpose(green_down)
    for i in lattice_linear_iterator
        doc = doc + green_up_c[i,i] * green_down_c[i,i]
    end
    doc / N_site
end

function structure_factor(green_up, green_down, latttice_iterator, latttice_linear_iterator)
    struct_factor = 0
    green_up_c = LinearAlgebra.I - transpose(green_up)
    green_down_c = LinearAlgebra.I - transpose(green_down)

    for site_i in latttice_linear_iterator
        for site_j in latttice_linear_iterator
            factor = 0
            Δr = latttice_iterator[site_i] - latttice_iterator[site_j]
            Δr_val = Δr.I[1] + Δr.I[2]
            if iseven(Δr_val)
                factor = 1
            else
                factor = -1
            end
            a = green_up_c[site_i,site_i] * green_up_c[site_j,site_j] + green_up_c[site_i,site_j] * green_up[site_i,site_j] +
                green_down_c[site_i,site_i] * green_down_c[site_j,site_j] + green_down_c[site_i,site_j] * green_down[site_i,site_j] -
                green_down_c[site_i,site_i] * green_up_c[site_j,site_j] - green_up_c[site_i,site_i] * green_down_c[site_j,site_j]
            struct_factor = struct_factor + factor * a
        end
    end
    struct_factor / (4 * N_site)
end

structure_factor (generic function with 1 method)

## Example Run

Define the `main` funciton.

In [21]:
function main()
    global α = acosh(exp(0.5 * Δτ * U))
    global lat_list = init_lattice_list(time_slices, l)
    global lat_iter = CartesianIndices(lat_list[1]) # Lattice iterator in Cartesian Coordinate.
    global lat_linear_iter = LinearIndices(lat_list[1]) # Lattice iterator in Linear Coordinate.
    global nil = gen_neighbor_list(lat_list[1], l)
    global S_list = Spin_Diag_list(time_slices, lat_list)
    global T_mat = T(t, l, lat_list[1], nil)
    global exp_A_T_mat = exp(-Δτ * t * T_mat)
    global exp_mA_T_mat = exp(Δτ * t * T_mat)
    global green_up_list = Array[]
    global green_down_list = Array[]
    global updated = false 

    global counts = Int[]
    push!(counts, 0) # Updated times
    push!(counts, 0) # Total times

    global green_up = zeros(N_site, N_site)
    global green_down = zeros(N_site, N_site)

    global B_up_mat_list = Array[]
    global B_down_mat_list = Array[]

    global E_K_list = Float64[]
    global DOC_list = Float64[]
    global SF_list = Float64[]

    global E_K_sum = 0
    global DOC_sum = 0
    global SF_sum = 0

    for time_index = 1:time_slices
        B_up_mat = B_up(α, S_list[time_index], exp_A_T_mat)
        B_down_mat = B_down(α, S_list[time_index], exp_A_T_mat)
        push!(B_up_mat_list, B_up_mat)
        push!(B_down_mat_list, B_down_mat)
    end

    # Initialize the Green function in each imaginary time slice with zero.
    for time_index = 1:time_slices
        push!(green_down_list, zeros(N_site, N_site))
        push!(green_up_list, zeros(N_site, N_site))
    end

    # Warm up sweep.
    for i = 1:N_warm_up_sweep
        print("$i ")
        measurement = false
        sweep(time_slices, green_up_list, green_down_list, B_up_mat_list, B_down_mat_list, lat_list, S_list, α, lat_iter, lat_linear_iter, measurement)
    end

    println("Finished warm up.")
    println(counts)

    for i = 1:N_bin
        E_K_sum = 0
        DOC_sum = 0
        SF_sum = 0
        for j = 1:N_sweep_per_bin
            # println("$j")
            measurement = true
            sweep(time_slices, green_up_list, green_down_list, B_up_mat_list, B_down_mat_list, lat_list, S_list, α, lat_iter, lat_linear_iter, measurement)
        end
        # println("bin $i finished.")
        push!(E_K_list, E_K_sum / N_sweep_per_bin)
        push!(DOC_list, DOC_sum / N_sweep_per_bin)
        push!(SF_list, SF_sum / N_sweep_per_bin)
        println(U, "\t", E_K_list[end], "\t", DOC_list[end], "\t", SF_list[end] / N_site)
    end
    println("E_K Mean: ", Statistics.mean(E_K_list))
    println("E_K Std: ", Statistics.std(E_K_list))
    println("Double Occupancy Mean: ", Statistics.mean(DOC_list))
    println("Double Occupancy Std: ", Statistics.std(DOC_list))
    println("Structure Factor Mean: ", Statistics.mean(SF_list / N_site))
    println("Structure Factor Std: ", Statistics.std(SF_list / N_site))
end


main (generic function with 1 method)

Run `main` function, this is where the program acctually start.

In [22]:
main()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 Finished warm up.
[306404, 327680]
1.0	-1.4835724549428293	0.21708558221003532	0.015